# 01 - Battery Data Retrieval from InfluxDB

This notebook retrieves battery monitoring data from the InfluxDB database and saves it locally for further processing. The data includes metrics such as battery temperature, voltage, current, state of charge, and other operational parameters.


## Install Required Dependencies

Install the necessary Python packages for working with InfluxDB and data manipulation.


In [ ]:
!pip install pandas influxdb-client

## Configure InfluxDB Connection

Set up the connection parameters to connect to the InfluxDB instance where battery data is stored. 

**IMPORTANT!** *Make sure the `INFLUXDB_URL` provided corresponds to the one for your InfluxDB database.*

In [6]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

# IMPORTANT! Replace with your InfluxDB instance URL
INFLUXDB_URL = "https://influx-db-microshift-001.apps.cluster-khrkg.dynamic.redhatworkshops.io/" 
INFLUXDB_TOKEN = "admin_token"
INFLUXDB_ORG = "redhat"
INFLUXDB_BUCKET = "bms"

## Connect to InfluxDB and Define Data Retrieval Function

Establish the connection to InfluxDB and create a function to query battery data from the last hour.


In [7]:
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)


def retrieve_battery_data():
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -1h)
      |> filter(fn: (r) => r["_measurement"] == "battery_data")
    '''
    query_api = client.query_api()
    tables = query_api.query(query, org=INFLUXDB_ORG)

    data = []
    for table in tables:
        for record in table.records:
            data.append(record.values)

    df = pd.DataFrame(data)
    return df

## Retrieve and Save Battery Data

Execute the query to retrieve battery data, clean up unnecessary columns, and save the raw data to a CSV file.


In [9]:
df = retrieve_battery_data()
df = df.drop(columns=["result", "table", "_measurement"])
df.to_csv("../data/battery_raw.csv", index=False)

## Preview Retrieved Data

Display the first few rows and basic information about the retrieved battery data to verify the retrieval was successful.


In [10]:
print(f"Retrieved {len(df)} rows of battery data")
df.head(10)

Retrieved 3987 rows of battery data


,_start,_stop,_time,_value,_field,batteryId
0,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:28.338000+00:00,19.47,ambientTemp,1
1,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:31.344000+00:00,19.37,ambientTemp,1
2,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:34.334000+00:00,18.98,ambientTemp,1
3,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:37.337000+00:00,18.68,ambientTemp,1
4,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:40.339000+00:00,18.74,ambientTemp,1
5,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:43.339000+00:00,18.76,ambientTemp,1
6,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:46.341000+00:00,19.09,ambientTemp,1
7,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:49.345000+00:00,19.33,ambientTemp,1
8,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:52.335000+00:00,18.96,ambientTemp,1
9,2026-01-04 19:22:06.889736+00:00,2026-01-04 20:22:06.889736+00:00,2026-01-04 19:52:55.335000+00:00,19.19,ambientTemp,1
